In [2]:
#!g1.1
%pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadat

In [3]:
#!g1.1
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
import torch
import nltk
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.optim import Adam
from string import punctuation as punct
from torch.utils.data import DataLoader
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
device = torch.device('cuda:0')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [26]:
#!g1.1
#train_data = pd.read_csv('train_all.tsv', sep='\t')
#val_df = pd.read_csv('data/data/val_empty.tsv', sep='\t')
test_df = pd.read_csv('test-no_labels.tsv', sep='\t')

In [27]:
#!g1.1
tokenizer_tr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model_tr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
model_tr.to(device)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62518, 512, padding_idx=62517)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62518, 512, padding_idx=62517)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [ ]:
#!g1.1

tr_tr = train_data.copy()

for i in tqdm(range(tr_tr.shape[0])):
    batch = tokenizer_tr(tr_tr['text'][i], return_tensors="pt").to(device)
    generated_ids = model_tr.generate(**batch)
    tr_text = tokenizer_tr.batch_decode(generated_ids, skip_special_tokens=True)[0]
    tr_tr.at[i, 'text'] = tr_text

  0%|          | 0/6717 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [28]:
#!g1.1
test_tr = test_df.copy()

for i in tqdm(range(test_tr.shape[0])):
    batch1 = tokenizer_tr(test_tr['text'][i], return_tensors="pt").to(device)
    generated_ids1 = model_tr.generate(**batch1)
    tr_text1 = tokenizer_tr.batch_decode(generated_ids1, skip_special_tokens=True)[0]
    test_tr.at[i, 'text'] = tr_text1

  0%|          | 0/1402 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [30]:
#!g1.1
def preprocessing(text):
    stop_words = set(stopwords.words('english'))
    numbers = '1234567890'
    clear = [sign for sign in text if sign not in punct]
    clear = [sign for sign in text if sign not in numbers]
    clear = ''.join(clear)
    tok_clear = word_tokenize(clear)
    prep = ' '.join([word for word in tok_clear if word.lower() not in stop_words])
    return prep

In [31]:
#!g1.1
test_tr['text'] = test_tr['text'].apply(preprocessing)
tr_tr['text'] = tr_tr['text'].apply(preprocessing)

In [ ]:
#!g1.1
train_df = tr_tr.sample(frac = 0.9)
val_df = tr_tr.drop(train_df.index)
print(train_df.shape, val_df.shape)

(6045, 8) (672, 8)


In [ ]:
train_df.to_csv('train.csv')
val_df.to_csv('val.csv')

In [48]:
test_tr.to_csv('test.csv')

In [7]:
train_df = pd.read_csv('train.csv')
val_df = pd.read_csv('val.csv')


In [32]:
test_tr

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17059,"Vacation would give rise spread virus , weeken...",NaN,NaN,NaN,NaN,NaN,NaN
1,17072,"Thought 'd sit little company , talk everyone ...",NaN,NaN,NaN,NaN,NaN,NaN
2,17077,"[ USER ] , China , disease way since December ...",NaN,NaN,NaN,NaN,NaN,NaN
3,17082,"unimptomably good third population sick , ever...",NaN,NaN,NaN,NaN,NaN,NaN
4,17089,training involves mass accumulation people clo...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1397,33948,"'s interesting thing , 's better sure health ,...",NaN,NaN,NaN,NaN,NaN,NaN
1398,33957,"vaccine options promised released soon , expec...",NaN,NaN,NaN,NaN,NaN,NaN
1399,33960,"Soon , vaccine disease become available , inoc...",NaN,NaN,NaN,NaN,NaN,NaN
1400,33993,"difficulties , course , end year , everyone wa...",NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert-v2")

model = AutoModel.from_pretrained("digitalepidemiologylab/covid-twitter-bert-v2")

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert-v2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
model = model.to(device)

In [ ]:
#!g1.1
new = train_df.reset_index(drop=True)

In [12]:
new = new.replace(-1, 3)

In [ ]:
new_val = val_df.reset_index(drop = True)

In [14]:
new_val = new_val.replace(-1, 3)

In [16]:
#!g1.1
embs = []
for i in range(new['text'].shape[0]):
    tokenized_text = tokenizer(new['text'][i], padding='max_length', truncation= True, max_length=42, return_tensors="pt").to(device)
    with torch.no_grad():
        x = model(**tokenized_text)
    embs.append(torch.nn.functional.normalize(x.last_hidden_state[:,:,:]).squeeze())

In [17]:
embs_val = []
for i in range(new_val.shape[0]):
    tokenized_text = tokenizer(new_val['text'][i], padding='max_length', truncation= True, max_length=42, return_tensors="pt").to(device)
    with torch.no_grad():
        x = model(**tokenized_text)
    embs_val.append(torch.nn.functional.normalize(x.last_hidden_state[:,:,:]).squeeze())


In [33]:
embs_test = []
for i in range(test_tr.shape[0]):
    tokenized_text = tokenizer(test_tr['text'][i], padding='max_length', truncation= True, max_length=42, return_tensors="pt").to(device)
    with torch.no_grad():
        x = model(**tokenized_text)
    embs_test.append(torch.nn.functional.normalize(x.last_hidden_state[:,:,:]).squeeze())

In [36]:
embs_test_fin = torch.stack([tensor for tensor in embs_test])

In [18]:
#!g1.1
class Dataset(torch.utils.data.Dataset):

  def __init__(self, embs, new, label):

        self.labels = new[label]
        self.embs = embs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.embs)

  def __getitem__(self, index):
        
        X = self.embs[index]
        y = self.labels[index]

        return X, torch.tensor(y)

In [19]:
new = new.drop(['Unnamed: 0', 'text_id', 'text'], axis = 1)
new_val = new_val.drop(['Unnamed: 0', 'text_id', 'text'], axis = 1)

In [ ]:
new_val.columns

Index(['masks_stance', 'masks_argument', 'quarantine_stance',
       'quarantine_argument', 'vaccines_stance', 'vaccines_argument'],
      dtype='object')

In [20]:
#!g1.1
train_loader = {}
for label in new.columns:
  dataset = Dataset(embs, new, label = label)
  train_loader[f'{label}'] = DataLoader(dataset, batch_size = 32, shuffle = False)

In [21]:
test_loader = {}
for label in new_val.columns:
  dataset = Dataset(embs_val, new_val, label = label)
  test_loader[f'{label}'] = DataLoader(dataset, batch_size = 32, shuffle = False)

In [69]:
#!g1.1
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            
            nn.Flatten(),
            nn.BatchNorm1d(42*1024),
            nn.Linear(42*1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 4)
        )

    def forward(self, x):
        return self.layers(x)

In [23]:
#!g1.1
def epoch_train(loader, clf, criterion, opt):
    # BEGIN Solution (do not delete this comment!)

    clf.train(True)

    avg_loss, avg_accuracy = 0, 0

    for i, data in enumerate(loader):
      x, y = data
      x, y = x.to(device), y.to(device)

      opt.zero_grad()
      predicts = clf(x)
      loss = criterion(predicts, y)
      loss.backward()
      opt.step()

      avg_loss += loss.item()
      cl_pred = torch.argmax(predicts, dim=1)
      avg_accuracy += (cl_pred == y).sum().item()

    return avg_loss * len(x) / len(loader.dataset), avg_accuracy / len(loader.dataset)

    # END Solution (do not delete this comment!)

def epoch_test(loader, clf, criterion):
    # BEGIN Solution (do not delete this comment!)

    clf.eval()

    avg_loss, avg_accuracy = 0, 0

    for i, data in enumerate(loader):
      x, y = data
      x, y = x.to(device), y.to(device)
 
      predicts = clf(x)
      loss = criterion(predicts, y)
      avg_loss += loss.item()
      cl_pred = torch.argmax(predicts, dim=1)
      avg_accuracy += (cl_pred == y).sum().item()

    return avg_loss * len(x) / len(loader.dataset), avg_accuracy / len(loader.dataset)

    # END Solution (do not delete this comment!)

# The function which you are going to use for model training
def train(train_loader, test_loader, clf, criterion, opt, n_epochs=10):
    for epoch in tqdm(range(n_epochs)):
        train_loss, train_acc = epoch_train(train_loader, clf, criterion, opt)
        test_loss, test_acc = epoch_test(test_loader, clf, criterion)

        print(f'[Epoch {epoch + 1}] train loss: {train_loss:.3f}; train acc: {train_acc:.2f}; ' + 
              f'test loss: {test_loss:.3f}; test acc: {test_acc:.2f}')

In [24]:
#!g1.1
clf_mlp = MLP().to(device)
opt =  torch.optim.Adam(clf_mlp.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [71]:
for label in new.columns:
  clf_mlp = MLP().to(device)
  opt =  torch.optim.Adam(clf_mlp.parameters(), lr=0.001)
  criterion = nn.CrossEntropyLoss()
  train(train_loader[label], test_loader[label], clf_mlp, criterion, opt, n_epochs=50)
  preds = clf_mlp(embs_test_fin)
  preds_fin = torch.argmax(preds, dim=1)
  test_tr[label] = preds_fin.cpu()


  0%|          | 0/50 [00:00<?, ?it/s]

[Epoch 1] train loss: 0.512; train acc: 0.77; test loss: 0.439; test acc: 0.83
[Epoch 2] train loss: 0.289; train acc: 0.88; test loss: 0.535; test acc: 0.79
[Epoch 3] train loss: 0.121; train acc: 0.95; test loss: 0.705; test acc: 0.81
[Epoch 4] train loss: 0.089; train acc: 0.97; test loss: 0.707; test acc: 0.81
[Epoch 5] train loss: 0.067; train acc: 0.97; test loss: 0.756; test acc: 0.81
[Epoch 6] train loss: 0.046; train acc: 0.98; test loss: 0.759; test acc: 0.82
[Epoch 7] train loss: 0.028; train acc: 0.99; test loss: 0.873; test acc: 0.82
[Epoch 8] train loss: 0.014; train acc: 1.00; test loss: 0.908; test acc: 0.82
[Epoch 9] train loss: 0.021; train acc: 0.99; test loss: 0.944; test acc: 0.80
[Epoch 10] train loss: 0.025; train acc: 0.99; test loss: 0.941; test acc: 0.82
[Epoch 11] train loss: 0.027; train acc: 0.99; test loss: 1.018; test acc: 0.81
[Epoch 12] train loss: 0.028; train acc: 0.99; test loss: 1.004; test acc: 0.81
[Epoch 13] train loss: 0.028; train acc: 0.99; te

  0%|          | 0/50 [00:00<?, ?it/s]

[Epoch 1] train loss: 0.375; train acc: 0.86; test loss: 0.317; test acc: 0.88
[Epoch 2] train loss: 0.174; train acc: 0.93; test loss: 0.403; test acc: 0.85
[Epoch 3] train loss: 0.073; train acc: 0.97; test loss: 0.469; test acc: 0.86
[Epoch 4] train loss: 0.051; train acc: 0.98; test loss: 0.461; test acc: 0.88
[Epoch 5] train loss: 0.041; train acc: 0.98; test loss: 0.479; test acc: 0.87
[Epoch 6] train loss: 0.024; train acc: 0.99; test loss: 0.554; test acc: 0.88
[Epoch 7] train loss: 0.020; train acc: 0.99; test loss: 0.624; test acc: 0.87
[Epoch 8] train loss: 0.008; train acc: 1.00; test loss: 0.602; test acc: 0.88
[Epoch 9] train loss: 0.007; train acc: 1.00; test loss: 0.622; test acc: 0.88
[Epoch 10] train loss: 0.014; train acc: 1.00; test loss: 0.647; test acc: 0.88
[Epoch 11] train loss: 0.016; train acc: 0.99; test loss: 0.606; test acc: 0.89
[Epoch 12] train loss: 0.026; train acc: 0.99; test loss: 0.626; test acc: 0.88
[Epoch 13] train loss: 0.020; train acc: 0.99; te

  0%|          | 0/50 [00:00<?, ?it/s]

...

[Epoch 1] train loss: 0.279; train acc: 0.91; test loss: 0.232; test acc: 0.93
[Epoch 2] train loss: 0.122; train acc: 0.96; test loss: 0.310; test acc: 0.92
[Epoch 3] train loss: 0.052; train acc: 0.98; test loss: 0.371; test acc: 0.93
[Epoch 4] train loss: 0.023; train acc: 0.99; test loss: 0.405; test acc: 0.93
[Epoch 5] train loss: 0.028; train acc: 0.99; test loss: 0.384; test acc: 0.93
[Epoch 6] train loss: 0.019; train acc: 0.99; test loss: 0.477; test acc: 0.92
[Epoch 7] train loss: 0.016; train acc: 0.99; test loss: 0.442; test acc: 0.93
[Epoch 8] train loss: 0.008; train acc: 1.00; test loss: 0.503; test acc: 0.93
[Epoch 9] train loss: 0.010; train acc: 1.00; test loss: 0.522; test acc: 0.93
[Epoch 10] train loss: 0.022; train acc: 0.99; test loss: 0.464; test acc: 0.94
[Epoch 11] train loss: 0.017; train acc: 0.99; test loss: 0.454; test acc: 0.93
[Epoch 12] train loss: 0.010; train acc: 1.00; test loss: 0.502; test acc: 0.93
[Epoch 13] train loss: 0.003; train acc: 1.00; te

  0%|          | 0/50 [00:00<?, ?it/s]

[Epoch 1] train loss: 0.350; train acc: 0.85; test loss: 0.294; test acc: 0.86
[Epoch 2] train loss: 0.212; train acc: 0.91; test loss: 0.350; test acc: 0.85
[Epoch 3] train loss: 0.106; train acc: 0.96; test loss: 0.446; test acc: 0.84
[Epoch 4] train loss: 0.069; train acc: 0.98; test loss: 0.426; test acc: 0.87
[Epoch 5] train loss: 0.042; train acc: 0.98; test loss: 0.551; test acc: 0.87
[Epoch 6] train loss: 0.027; train acc: 0.99; test loss: 0.525; test acc: 0.87
[Epoch 7] train loss: 0.019; train acc: 0.99; test loss: 0.506; test acc: 0.88
[Epoch 8] train loss: 0.025; train acc: 0.99; test loss: 0.541; test acc: 0.89
[Epoch 9] train loss: 0.014; train acc: 1.00; test loss: 0.616; test acc: 0.88
[Epoch 10] train loss: 0.017; train acc: 0.99; test loss: 0.643; test acc: 0.87
[Epoch 11] train loss: 0.021; train acc: 0.99; test loss: 0.613; test acc: 0.88
[Epoch 12] train loss: 0.010; train acc: 1.00; test loss: 0.670; test acc: 0.88
[Epoch 13] train loss: 0.012; train acc: 1.00; te

  0%|          | 0/50 [00:00<?, ?it/s]

[Epoch 1] train loss: 0.284; train acc: 0.90; test loss: 0.215; test acc: 0.92
[Epoch 2] train loss: 0.143; train acc: 0.94; test loss: 0.273; test acc: 0.91
[Epoch 3] train loss: 0.065; train acc: 0.98; test loss: 0.364; test acc: 0.90
[Epoch 4] train loss: 0.042; train acc: 0.99; test loss: 0.414; test acc: 0.91
[Epoch 5] train loss: 0.035; train acc: 0.99; test loss: 0.436; test acc: 0.92
[Epoch 6] train loss: 0.026; train acc: 0.99; test loss: 0.454; test acc: 0.93
[Epoch 7] train loss: 0.015; train acc: 0.99; test loss: 0.429; test acc: 0.93
[Epoch 8] train loss: 0.010; train acc: 1.00; test loss: 0.436; test acc: 0.92
[Epoch 9] train loss: 0.009; train acc: 1.00; test loss: 0.464; test acc: 0.92
[Epoch 10] train loss: 0.008; train acc: 1.00; test loss: 0.494; test acc: 0.92
[Epoch 11] train loss: 0.007; train acc: 1.00; test loss: 0.461; test acc: 0.93
[Epoch 12] train loss: 0.014; train acc: 0.99; test loss: 0.522; test acc: 0.90
[Epoch 13] train loss: 0.011; train acc: 1.00; te

In [72]:
sub = test_tr.drop(['text_id', 'text'], axis = 1)

In [73]:
sub = sub.replace(3, -1)

In [75]:
sub

,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,-1,-1,1,1,-1,-1
1,-1,-1,1,1,-1,-1
2,-1,-1,2,1,-1,-1
3,-1,-1,1,1,-1,-1
4,-1,-1,1,1,-1,-1
...,...,...,...,...,...,...
1397,-1,-1,-1,-1,0,1
1398,-1,-1,-1,-1,2,-1
1399,2,2,-1,-1,2,0
1400,-1,-1,-1,-1,2,1


In [76]:
sub.to_csv('finally.tsv', index=False, sep='\t')
!zip finally.zip finally.tsv

updating: finally.tsv (deflated 95%)


In [25]:
train(train_loader['masks_argument'], test_loader['masks_argument'], clf_mlp, criterion, opt, n_epochs=5)

  0%|          | 0/5 [00:00<?, ?it/s]

[Epoch 1] train loss: 0.338; train acc: 0.88; test loss: 0.285; test acc: 0.90
[Epoch 2] train loss: 0.122; train acc: 0.95; test loss: 0.325; test acc: 0.88
[Epoch 3] train loss: 0.061; train acc: 0.98; test loss: 0.495; test acc: 0.89
[Epoch 4] train loss: 0.030; train acc: 0.99; test loss: 0.604; test acc: 0.86
[Epoch 5] train loss: 0.028; train acc: 0.99; test loss: 0.512; test acc: 0.87


In [39]:
preds = clf_mlp(embs_test_fin)

In [45]:
preds[0:50]

tensor([[-10.2507,  -4.5157, -10.9899,  24.1776],
        [ -5.3558,  -3.2515,  -6.8107,  14.7656],
        [ -4.8072,  -2.9995,  -4.8204,  12.2483],
        [ -5.4218,  -3.6477,  -4.5143,  13.6083],
        [ -3.9280,  -2.9709,  -5.2924,  12.0927],
        [ -7.4521,  -0.8171, -10.1114,  17.5492],
        [ -3.3757,  -3.7359,  -4.7902,  11.4773],
        [ -7.5759,  -2.8296,  -8.6759,  18.3317],
        [ -5.3404,  -2.0210,  -5.5151,  12.8354],
        [ -9.3888,  -2.2368, -12.1148,  22.5108],
        [ -3.0140,  -1.4403,  -4.0104,   8.2203],
        [ -1.5004,  -0.1682,  -7.3594,   9.3151],
        [ -5.3710,  -0.3781,  -6.5713,  12.3470],
        [ -6.8325,  -4.0628,  -7.1901,  17.6217],
        [ -4.5239,  -1.3021,  -6.5197,  12.0692],
        [ -8.4957,  -1.4170, -10.4022,  18.7345],
        [ -8.9514,  -4.7702, -10.4399,  23.1154],
        [ -6.4403,   0.2145,  -6.8413,  12.7963],
        [ -4.7681,  -2.8981,  -7.2704,  14.5975],
        [ -9.2381,  -4.2697,  -8.6579,  20.9146],


In [41]:
preds_fin = torch.argmax(preds, dim=1)

In [56]:
preds_fin

tensor([3, 3, 3,  ..., 2, 3, 3], device='cuda:0')

In [54]:
test_tr['masks_argument'] = preds_fin.cpu()

In [55]:
test_tr

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17059,"Vacation would give rise spread virus , weeken...",NaN,3,NaN,NaN,NaN,NaN
1,17072,"Thought 'd sit little company , talk everyone ...",NaN,3,NaN,NaN,NaN,NaN
2,17077,"[ USER ] , China , disease way since December ...",NaN,3,NaN,NaN,NaN,NaN
3,17082,"unimptomably good third population sick , ever...",NaN,3,NaN,NaN,NaN,NaN
4,17089,training involves mass accumulation people clo...,NaN,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1397,33948,"'s interesting thing , 's better sure health ,...",NaN,3,NaN,NaN,NaN,NaN
1398,33957,"vaccine options promised released soon , expec...",NaN,3,NaN,NaN,NaN,NaN
1399,33960,"Soon , vaccine disease become available , inoc...",NaN,2,NaN,NaN,NaN,NaN
1400,33993,"difficulties , course , end year , everyone wa...",NaN,3,NaN,NaN,NaN,NaN


In [ ]:
clf_mlp.train(True)

avg_loss, avg_accuracy = 0, 0

#for i, data in enumerate(train_loader):
  #x, y = data
#x, y = x.to('cuda'), y.to('cuda')
# x = torch.tensor(x, dtype=torch.float32)
# y = torch.tensor(y, dtype=torch.float32)
opt.zero_grad()
predicts = clf_mlp(x)
loss = criterion(predicts, y)
loss.backward()
opt.step()


In [ ]:
torch.argmax(predicts, dim = 1).dtype

torch.int64

In [ ]:
predicts.shape

NameError: ignored

In [ ]:
x.shape


torch.Size([4, 42, 1024])

In [ ]:
for i in range(y.shape[0]):
  y[i]= 1


In [ ]:
y

tensor([1, 1, 1, 1])

In [ ]:
train_df

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
2325,22963,"says every door , wear masks .",1,1,-1,-1,-1,-1
1016,19662,"'Cause get test , quarantine 's .",-1,-1,1,1,-1,-1
3093,24828,'s understandable masks really working .,2,1,-1,-1,-1,-1
154,17412,quarantine ) ) ... percent alcohol released lu...,-1,-1,1,1,-1,-1
4426,28237,"Try think : children vectors , walk without ma...",1,1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
4576,28602,masks wholesale r producer Svoplivius direct p...,1,1,-1,-1,-1,-1
4598,28647,Better wear mask IVL machine .,2,2,-1,-1,-1,-1
431,18167,"[ USER ] , people suffering mild form virus in...",-1,-1,2,2,-1,-1
5,17033,"[ USER ] , many home quarantine .",-1,-1,1,1,-1,-1


In [ ]:
val_df['masks_stance'].values

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  0, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  2,  2,  1,  1,  1,
        0,  1,  2,  2,  1

In [ ]:
tr_tr

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17024,"[ USER ] According Rosspernadzor , transit pas...",-1,-1,1,1,-1,-1
1,17025,Non-compliance quarantine measures contact per...,-1,-1,1,1,-1,-1
2,17027,"[ USER ] , read books quarantine , see resusci...",-1,-1,1,1,-1,-1
3,17030,Go read intuition city pavilions one ten says ...,-1,-1,1,1,-1,-1
4,17031,contact persons identified sent -day quarantin...,-1,-1,1,1,-1,-1
...,...,...,...,...,...,...,...,...
6712,34033,"[ USER ] , 'm concerned doctors forced pack sc...",-1,-1,-1,-1,2,1
6713,34034,Eats eats virus people vaccine comes us ?,-1,-1,-1,-1,2,1
6714,34035,wondering else would kill vaccine carpet ?,-1,-1,-1,-1,1,1
6715,34037,"'re gon na talk `` Russian medicine , '' less ...",-1,-1,-1,-1,2,1


In [ ]:
torch.argmax(predicts[1])

tensor(3, device='cuda:0')

In [ ]:
#!g1.1
train(train_loader, clf_mlp, criterion, opt, n_epochs=115)

/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [6,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [8,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [9,0,0] Assertion `t >= 0 

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:912: UserWarning: The following variables cannot be serialized: batch, clf, clf_mlp, data, dataset, embsar, model, modelclf, opt, optimizer, pbar, tokenized_text, train_loader, x
  warnings.warn(message)


In [ ]:
#!g1.1
